In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("udf-review").getOrCreate()

22/04/05 16:19:15 WARN Utils: Your hostname, devkhk-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 172.30.1.27 instead (on interface en0)
22/04/05 16:19:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/05 16:19:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
transactions = [
    ('찹쌀탕수육+짜장2', '2021-11-07 13:20:00', 22000, 'KRW'),
    ('등심탕수육+크립새우+짜장면', '2021-10-24 11:19:00', 21500, 'KRW'),
    ('월남 쌈 2인 세트', '2021-07-25 11:12:40', 42000, 'KRW'),
    ('콩국수+열무비빔국수', '2021-07-10 08:20:00', 21250, 'KRW'),
    ('장어소금+고추장구이', '2021-07-01 05:36:00', 68700, 'KRW'),
    ('족발', '2020-08-19 19:04:00', 32000, 'KRW'),
]

schema = ["name", "datetime", "price", "currency"]

In [4]:
df = spark.createDataFrame(data=transactions, schema=schema)

In [5]:
df.createOrReplaceTempView("transactions")

In [7]:
spark.sql("select * from transactions").show()

+--------------------------+-------------------+-----+--------+
|                      name|           datetime|price|currency|
+--------------------------+-------------------+-----+--------+
|          찹쌀탕수육+짜장2|2021-11-07 13:20:00|22000|     KRW|
|등심탕수육+크립새우+짜장면|2021-10-24 11:19:00|21500|     KRW|
|          월남 쌈 2인 세트|2021-07-25 11:12:40|42000|     KRW|
|       콩국수+열무비빔국수|2021-07-10 08:20:00|21250|     KRW|
|       장어소금+고추장구이|2021-07-01 05:36:00|68700|     KRW|
|                      족발|2020-08-19 19:04:00|32000|     KRW|
+--------------------------+-------------------+-----+--------+



In [8]:
spark.sql("select datetime, price from transactions").show()

+-------------------+-----+
|           datetime|price|
+-------------------+-----+
|2021-11-07 13:20:00|22000|
|2021-10-24 11:19:00|21500|
|2021-07-25 11:12:40|42000|
|2021-07-10 08:20:00|21250|
|2021-07-01 05:36:00|68700|
|2020-08-19 19:04:00|32000|
+-------------------+-----+



In [13]:
def transPrice(n):
    ko = ["", "십", "백", "천", "만"]
    num = "일이삼사오육칠팔구"
    result = []
    i = 0
    
    # 21250
    while n > 0:
        n, r = divmod(n, 10)
        if r > 0:
            result.append(num[r-1] + ko[i])
        i += 1
    return "".join(reversed(result))

In [15]:
transPrice(3200)

'삼천이백'

In [17]:
# udf 등록하기

spark.udf.register("transactions", transPrice)


<function __main__.transPrice(n)>

In [19]:
spark.sql("select datetime, price, transactions(price) from transactions").show()

+-------------------+-----+-------------------+
|           datetime|price|transactions(price)|
+-------------------+-----+-------------------+
|2021-11-07 13:20:00|22000|           이만이천|
|2021-10-24 11:19:00|21500|       이만일천오백|
|2021-07-25 11:12:40|42000|           사만이천|
|2021-07-10 08:20:00|21250|   이만일천이백오십|
|2021-07-01 05:36:00|68700|       육만팔천칠백|
|2020-08-19 19:04:00|32000|           삼만이천|
+-------------------+-----+-------------------+

